# Урок 6. Рекуррентные нейронные сети. LSTM. GRU.

ЗАДАНИЕ 1.

Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

In [2]:
import pickle
with open('combine_df.pickle', 'rb') as f:
    combine_df = pickle.load(f)
    
combine_df.head()

,index,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized,tweet_vecs
0,0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy...","[[0.40891476115211844, 0.18360024644061923, -0..."
1,1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee...","[[0.4508066247491276, 0.17692823379355319, -0...."
2,2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]","[[0.7542365392049154, 0.5224650899569193, 0.00..."
3,3,4,0.0,#model i love u take with u all the time in ...,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[[0.49165773598684204, 0.2370210364460945, -0...."
4,4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]","[[0.3752712719142437, 0.1220138180651702, -0.1..."


In [26]:
import pandas as pd

In [3]:
combine_df['label'].fillna(0,inplace=True)

In [6]:
len(combine_df[combine_df['label']>0])/len(combine_df)

0.04560711161740475

In [7]:
from sklearn.model_selection import train_test_split

In [62]:
df_train_x, df_val_x, df_train_y, df_val_y = train_test_split(combine_df['clean_tweet'], combine_df['label'], test_size=0.2)


In [63]:
len(df_train_y[df_train_y>0])/len(df_train_y)

0.045439519922699415

In [64]:
len(df_val_y[df_val_y>0])/len(df_val_y)

0.04627746135069162

In [65]:
df_train_x.head()

19249    well donald trump has officially reached the b...
30071           this looks like molly old boyfriend gunner
7869     weeks on this picture weeks on monday not long...
4087     these niggas burning will not let you know the...
2282                                             well done
Name: clean_tweet, dtype: object

In [66]:
type(df_train_x)

pandas.core.series.Series

In [67]:
ds_train_x=pd.DataFrame(data=df_train_x.values,index=df_train_x.index,columns=['text'])

In [68]:
ds_train_x.head()

,text
19249,well donald trump has officially reached the b...
30071,this looks like molly old boyfriend gunner
7869,weeks on this picture weeks on monday not long...
4087,these niggas burning will not let you know the...
2282,well done


In [69]:
df_val_x.head()

4925     fitcamp tonight at manupmonday letssss go heal...
19869           always love good friday selfie vain friday
34470    she seems to have split personality rav does t...
48729    isis is so weak call identify any man in front...
15460    operation jefferson jay covers bob ross remixe...
Name: clean_tweet, dtype: object

In [70]:
ds_val_x=pd.DataFrame(data=df_val_x.values,index=df_val_x.index,columns=['text'])

In [71]:
ds_val_x.head()

,text
4925,fitcamp tonight at manupmonday letssss go heal...
19869,always love good friday selfie vain friday
34470,she seems to have split personality rav does t...
48729,isis is so weak call identify any man in front...
15460,operation jefferson jay covers bob ross remixe...


In [73]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [74]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

ds_train_x['text'] = ds_train_x['text'].apply(preprocess_text)
ds_val_x['text'] = ds_val_x['text'].apply(preprocess_text)

In [76]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [77]:
text_corpus_train = ds_train_x['text'].values
text_corpus_valid = ds_val_x['text'].values

In [79]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)


In [80]:
y_train = df_train_y.values
y_val = df_val_y.values

**RNN model**

In [84]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [86]:
#early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=50,
                    verbose=1,
                    validation_split=0.1)#,
                    #callbacks=[early_stopping])

Epoch 1/50
70/70 [==============================] - 2s 32ms/step - loss: 0.0172 - accuracy: 0.9928 - val_loss: 0.1786 - val_accuracy: 0.9547
Epoch 2/50
70/70 [==============================] - 2s 32ms/step - loss: 0.0147 - accuracy: 0.9930 - val_loss: 0.1960 - val_accuracy: 0.9570
Epoch 3/50
70/70 [==============================] - 2s 33ms/step - loss: 0.0133 - accuracy: 0.9934 - val_loss: 0.2133 - val_accuracy: 0.9601
Epoch 4/50
70/70 [==============================] - 3s 48ms/step - loss: 0.0141 - accuracy: 0.9926 - val_loss: 0.2006 - val_accuracy: 0.9532
Epoch 5/50
70/70 [==============================] - 2s 32ms/step - loss: 0.0124 - accuracy: 0.9936 - val_loss: 0.2174 - val_accuracy: 0.9532
Epoch 6/50
70/70 [==============================] - 2s 30ms/step - loss: 0.0116 - accuracy: 0.9938 - val_loss: 0.2187 - val_accuracy: 0.9527
Epoch 7/50
70/70 [==============================] - 2s 32ms/step - loss: 0.0109 - accuracy: 0.9938 - val_loss: 0.2401 - val_accuracy: 0.9565
Epoch 8/50
70

In [87]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 0s 14ms/step - loss: 0.3810 - accuracy: 0.9410


Test score: 0.38096410036087036
Test accuracy: 0.9410089254379272


**LSTM model**

In [88]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [89]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=50,
                    verbose=1,
                    validation_split=0.1)
                   

Epoch 1/50
70/70 [==============================] - 8s 72ms/step - loss: 0.4191 - accuracy: 0.9271 - val_loss: 0.1681 - val_accuracy: 0.9593
Epoch 2/50
70/70 [==============================] - 4s 59ms/step - loss: 0.1842 - accuracy: 0.9549 - val_loss: 0.1205 - val_accuracy: 0.9606
Epoch 3/50
70/70 [==============================] - 4s 60ms/step - loss: 0.0960 - accuracy: 0.9640 - val_loss: 0.1157 - val_accuracy: 0.9560
Epoch 4/50
70/70 [==============================] - 4s 59ms/step - loss: 0.0616 - accuracy: 0.9745 - val_loss: 0.1345 - val_accuracy: 0.9575
Epoch 5/50
70/70 [==============================] - 4s 59ms/step - loss: 0.0452 - accuracy: 0.9831 - val_loss: 0.1672 - val_accuracy: 0.9593
Epoch 6/50
70/70 [==============================] - 4s 60ms/step - loss: 0.0370 - accuracy: 0.9861 - val_loss: 0.1781 - val_accuracy: 0.9588
Epoch 7/50
70/70 [==============================] - 4s 62ms/step - loss: 0.0308 - accuracy: 0.9893 - val_loss: 0.1913 - val_accuracy: 0.9466
Epoch 8/50
70

In [90]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 0s 15ms/step - loss: 0.5619 - accuracy: 0.9423


Test score: 0.5619032979011536
Test accuracy: 0.9423311352729797


**GRU model**

In [91]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [92]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=50,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/50
70/70 [==============================] - 7s 61ms/step - loss: 0.4690 - accuracy: 0.9251 - val_loss: 0.1495 - val_accuracy: 0.9593
Epoch 2/50
70/70 [==============================] - 4s 54ms/step - loss: 0.1434 - accuracy: 0.9554 - val_loss: 0.1198 - val_accuracy: 0.9570
Epoch 3/50
70/70 [==============================] - 4s 54ms/step - loss: 0.0762 - accuracy: 0.9697 - val_loss: 0.1266 - val_accuracy: 0.9527
Epoch 4/50
70/70 [==============================] - 4s 54ms/step - loss: 0.0522 - accuracy: 0.9803 - val_loss: 0.1481 - val_accuracy: 0.9547
Epoch 5/50
70/70 [==============================] - 4s 55ms/step - loss: 0.0364 - accuracy: 0.9861 - val_loss: 0.1737 - val_accuracy: 0.9535
Epoch 6/50
70/70 [==============================] - 4s 54ms/step - loss: 0.0299 - accuracy: 0.9886 - val_loss: 0.2183 - val_accuracy: 0.9565
Epoch 7/50
70/70 [==============================] - 4s 55ms/step - loss: 0.0268 - accuracy: 0.9885 - val_loss: 0.2250 - val_accuracy: 0.9535
Epoch 8/50
70

In [93]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 0s 14ms/step - loss: 0.5122 - accuracy: 0.9423


Test score: 0.512185275554657
Test accuracy: 0.9423311352729797


**Вывод - немного лучше и одинаково работают LSTM и  GRU**

ЗАДАНИЕ 2

Провести анализ как лучше генерировать текст по словам или символам ноутбук text_generation.ipynb

In [70]:
import tensorflow as tf

import numpy as np
import os
import time

In [71]:
path_to_file = 'evgenyi_onegin.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))
print(text[:250])


Length of text: 286984 characters
Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                       


**Генерация текста по символам**

In [72]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
#text_as_int, text, len(text_as_int), len(text)

131 unique characters


In [73]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [74]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [75]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [76]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [77]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [78]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [79]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [80]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [81]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [82]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           33536     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_6 (Dense)              (64, None, 131)           134275    
Total params: 4,106,115
Trainable params: 4,106,115
Non-trainable params: 0
_________________________________________________________________


In [83]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.880374


In [84]:
model.compile(optimizer='adam', loss=loss)

In [85]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [86]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
44/44 [==============================] - 50s 1s/step - loss: 3.8858
Epoch 2/10
44/44 [==============================] - 48s 1s/step - loss: 1.7990
Epoch 3/10
44/44 [==============================] - 49s 1s/step - loss: 1.6012
Epoch 4/10
44/44 [==============================] - 48s 1s/step - loss: 1.4548
Epoch 5/10
44/44 [==============================] - 48s 1s/step - loss: 1.3631
Epoch 6/10
44/44 [==============================] - 48s 1s/step - loss: 1.3171
Epoch 7/10
44/44 [==============================] - 49s 1s/step - loss: 1.2769
Epoch 8/10
44/44 [==============================] - 49s 1s/step - loss: 1.2560
Epoch 9/10
44/44 [==============================] - 50s 1s/step - loss: 1.2226
Epoch 10/10
44/44 [==============================] - 49s 1s/step - loss: 1.2052


In [87]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [88]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 256)            33536     
_________________________________________________________________
gru_7 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_7 (Dense)              (1, None, 131)            134275    
Total params: 4,106,115
Trainable params: 4,106,115
Non-trainable params: 0
_________________________________________________________________


In [89]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [90]:
print(generate_text(model, start_string=u"Уж месяц был "))

Уж месяц был силь
              . . . . . . " На ягоят свовка.
                    Оны, придет снае швой..... - деться, писхов,
                         Она пилска. луквезскийшо.
                          И удрамную Евгогий
                       На мнегда доль отни слов медить,
                            Тум педрурсцевуя капить {3}}
                     В жу ут писманялаоколком,
     Подетатся стаза,
                      Блаждука моден овщем
                       Кво дутяль оз раро ум.

                 


In [92]:
for el1,el2 in dataset:
    print(el1)
    print(el2)

tf.Tensor(
[[104   1 103 ... 113 116 104]
 [  1   1   1 ...   1   1   1]
 [110 129 103 ...  99 111   1]
 ...
 [126 108   1 ...   1 112  99]
 [  1   1   1 ...   1   1   1]
 [  1   1   1 ...   0   1   1]], shape=(64, 100), dtype=int32)
tf.Tensor(
[[  1 103 118 ... 116 104 111]
 [  1   1   1 ...   1   1   1]
 [129 103 107 ... 111   1 103]
 ...
 [108   1 102 ... 112  99 116]
 [  1   1   1 ...   1   1   1]
 [  1   1   1 ...   1   1   1]], shape=(64, 100), dtype=int32)
tf.Tensor(
[[127   9   9 ... 117 127   1]
 [  1   1   1 ...   1   1   1]
 [  1   1   1 ...   0   1   1]
 ...
 [  1   1   1 ...   1   1   1]
 [  1   1   1 ... 106 107 110]
 [  1   1   1 ...   0   1   1]], shape=(64, 100), dtype=int32)
tf.Tensor(
[[  9   9   9 ... 127   1 105]
 [  1   1   1 ...   1   1   1]
 [  1   1   1 ...   1   1   1]
 ...
 [  1   1   1 ...   1   1   1]
 [  1   1   1 ... 107 110 116]
 [  1   1   1 ...   1   1   1]], shape=(64, 100), dtype=int32)
tf.Tensor(
[[  1   1   1 ...   1   1   1]
 [115 104 123 ...   1 

**Генерация текста по словам**

In [173]:
import string
 
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [174]:
# clean document
tokens = clean_doc(text)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['александр', 'сергеевич', 'пушкин', 'евгений', 'онегин', 'роман', 'в', 'стихах', 'не', 'мысля', 'гордый', 'свет', 'забавить', 'вниманье', 'дружбы', 'возлюбя', 'хотел', 'бы', 'я', 'тебе', 'представить', 'залог', 'достойнее', 'тебя', 'достойнее', 'души', 'прекрасной', 'святой', 'исполненной', 'мечты', 'поэзии', 'живой', 'и', 'ясной', 'высоких', 'дум', 'и', 'простоты', 'но', 'так', 'и', 'быть', 'рукой', 'пристрастной', 'прими', 'собранье', 'пестрых', 'глав', 'полусмешных', 'полупечальных', 'простонародных', 'идеальных', 'небрежный', 'плод', 'моих', 'забав', 'бессонниц', 'легких', 'вдохновений', 'незрелых', 'и', 'увядших', 'лет', 'ума', 'холодных', 'наблюдений', 'и', 'сердца', 'горестных', 'замет', 'глава', 'первая', 'и', 'жить', 'торопится', 'и', 'чувствовать', 'спешит', 'кн', 'вяземский', 'i', 'мой', 'дядя', 'самых', 'честных', 'правил', 'когда', 'не', 'в', 'шутку', 'занемог', 'он', 'уважать', 'себя', 'заставил', 'и', 'лучше', 'выдумать', 'не', 'мог', 'его', 'пример', 'другим', 'наука',

In [175]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)    
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 22963


In [176]:
sequences[1]

'сергеевич пушкин евгений онегин роман в стихах не мысля гордый свет забавить вниманье дружбы возлюбя хотел бы я тебе представить залог достойнее тебя достойнее души прекрасной святой исполненной мечты поэзии живой и ясной высоких дум и простоты но так и быть рукой пристрастной прими собранье пестрых глав полусмешных полупечальных простонародных идеальных'

In [177]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [178]:
out_filename = 'EOnegin.txt'
save_doc(sequences, out_filename)

In [179]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'EOnegin.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [180]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential

In [181]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [182]:
vocab_size = len(tokenizer.word_index) + 1

In [183]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [184]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 50, 50)            425950    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_13 (Dense)             (None, 8519)              860419    
Total params: 1,437,269
Trainable params: 1,437,269
Non-trainable params: 0
_________________________________________________________________
None


In [185]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
180/180 [==============================] - 22s 105ms/step - loss: 8.5196 - accuracy: 0.0329
Epoch 2/10
180/180 [==============================] - 19s 107ms/step - loss: 7.8350 - accuracy: 0.0420
Epoch 3/10
180/180 [==============================] - 19s 106ms/step - loss: 7.6703 - accuracy: 0.0443
Epoch 4/10
180/180 [==============================] - 19s 107ms/step - loss: 7.5345 - accuracy: 0.0448
Epoch 5/10
180/180 [==============================] - 19s 107ms/step - loss: 7.3760 - accuracy: 0.0501
Epoch 6/10
180/180 [==============================] - 19s 107ms/step - loss: 7.2796 - accuracy: 0.0477
Epoch 7/10
180/180 [==============================] - 19s 108ms/step - loss: 7.1744 - accuracy: 0.0494
Epoch 8/10
180/180 [==============================] - 19s 107ms/step - loss: 7.0395 - accuracy: 0.0508
Epoch 9/10
180/180 [==============================] - 19s 107ms/step - loss: 6.8933 - accuracy: 0.0511
Epoch 10/10
180/180 [==============================] - 19s 107ms/step - l

In [186]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [187]:
seq_length = len(lines[0].split()) - 1

In [188]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [189]:
model = load_model('model.h5')

In [190]:
tokenizer = load(open('tokenizer.pkl', 'rb'))

In [191]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

гробовой своей доски все потеряв невозвратимо и все равно надежда им лжет детским лепетом своим viii татьяна любопытным взором на воск потопленный глядит он чудно вылитым узором ей чтото чудное гласит из блюда полного водою выходят кольцы чередою и вынулось колечко ей под песенку старинных дней там мужичкито все богаты гребут



In [192]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]

In [197]:
# predict probabilities for each word
yhat = np.argmax(model.predict(encoded), axis=-1)
#yhat = model.predict_classes(encoded, verbose=0)

In [199]:
out_word = ''
for word, index in tokenizer.word_index.items():
    if index in yhat:
        out_word = word
        break

In [200]:
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

In [201]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index in yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [202]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

i и в ней ни ним и в ней ни ним и в ней ни ним и в ней ни ним и в ней ни ним и в ней ни ним и в ней ни ним и в ней ни ним и в ней ни ним и в ней ни


**Вывод: сравнивая как лучше генерировать текст по словам или символам - лучше генерировать текст по символам, т.к. результат при одинаковом количестве эпох получается лучше.**